In [13]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import linmix
from astropy.table import Table
from astropy.table import Column
import astropy.io.ascii as ascii
from lifelines import KaplanMeierFitter
from matplotlib import rc
from matplotlib.ticker import MultipleLocator


rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

%matplotlib inline
#%matplotlib qt5

In [14]:
###########################
#        read data       #

#Taurus Tab with fluxes and disk mass
Tab=Table.read("Table.fit")      
#Taurus Tab with stellar masses (from the different pre-MS models)
Tab_Mass=Table.read("Table_Mass.fit")   
#Lupus Tab with fluxes and dust masses
Lup_Tab=Table.read('Lupus_Tab.fit')     
#Chamaeleon I Tab with fluxes and dust masses
ChaI_Tab_orig=Table.read('ChaI_Tab.fit')
ChaI_Tab=ChaI_Tab_orig.copy()
ChaI_Tab.remove_rows(36)      #removing source without measurement of Mgas
#Upper Scorpius Tab with fluxes and dust masses
UppSco_Tab=Table.read('UppSco_Tab.fit')

#Lupus Tab with gas masses
Mgas_Lup_orig=Table.read('Mgas_Lup.dat',format='ascii')
#Chamaeleon I Tab with fgas masses
Mgas_ChaI=ascii.read('Mgas_ChaI.tex',format='latex')


#######################
#  remove from Lupus gas tab the 20 obscured sources in Lupus (without stellar mass measurements)
Lup_Ms=Table.read('Lup_Ms.fit')
Lup_Ms.remove_rows([13,14,16,35,53]) #removing non observed sources

nnan=np.isnan(Lup_Ms['Mass'])        #20 obscured sources
notnan=np.logical_not(nnan)
 
index=np.where(nnan)
Mgas_Lup=Mgas_Lup_orig.copy()
Mgas_Lup.remove_rows(index)
#print Mgas_Lup['source']

#####################################
#  choose the analysis to perform: dust mass or gas mass?    #
#  and the survey: Taurus, Lupus, Chamaeleon I or Upper Scorpius?   #

#survey='Tau'   #Taurus
#survey='Lup'   #Lupus
#survey='ChaI'   #ChamaleonI
#survey='UppSco'   #Upper Scorpius OB
survey='LupChaI'   #merged sample from Lup and ChaI data

#n='Mdust'     #Mdust
#n='Mgas'     #Mgas
n='g/d'     #gas to dust ratio

### choose only with gas-to-dust radio   ###
par='Ms'
    #### and only with Lupus   ####
#par='Rout'     #Outer dust radius
#par='Rgas'     #Gas radius
#par='spidx70'    # IR spectral index a 70 mum
#par='spidx100'    # IR spectral index a 100 mum
#par='spidx160'    # IR spectral index a 160 mum


In [27]:
##############################################################
###   Set x and y data and define the upperlimits      ####
## x -->  Stellar masses from Siess+2000 pre-MS model (except for Chamaeleon, we used Baraffe+15), 
#         limited to exlude brown dwarfs (M*>0.1 Msun)#
## y --> n=2 dust mass, computed with a constant dust temperature T=20;  in *Eart Mass* unit #
## y --> n=3 gas mass, computed from the CO and its isotopologue lines, using the Miotello et al. (2016) models.  
#        in *Solar mass* unit #
##  Introduction of variable delta --> if delta is True we have an observed data, 
#                                      if it is False we have an upper limit.

if survey=='Tau':
    limMs=10**Tab_Mass['LogM*3']>=0.1
    if n=='Mdust':               
        x=Tab_Mass['LogM*3'][limMs]     
        y=Tab['LogMd_20'][limMs]     
        delta=Tab['l'][limMs]!='<'            #observed sources  
        
if survey=='Lup':
    if n=='Mdust': 
        x=np.log10(Lup_Tab['M*'])
        delta=Lup_Tab['l_Md']!='<'      #observed sources for Mdust
        Md=Lup_Tab['Md']
        e_Md=Lup_Tab['err_Md']  
        notdelta = np.logical_not(delta)       #upperlimits
        Md[notdelta]=3*e_Md[notdelta]    # upperlimits as 3*sigma   
        y=np.log10(Md)  

    if n=='Mgas':                  
        x=np.log10(Lup_Tab['M*'])
        y=Mgas_Lup['logMgas']   
        delta=Mgas_Lup['l_Mgas']!='<'      #observed sources for Mgas
        
    if n=='g/d':
        delta_dust=Lup_Tab['l_Md']!='<'     
        notdelta_dust=np.logical_not(delta_dust)
        dGas=Mgas_Lup['l_Mgas']!='<'      #observed sources for Mgas
        delta=dGas[delta_dust]          # take only the sources detected in dust
        ratio=10**Mgas_Lup['logMgas'][delta_dust]/(Lup_Tab['Md'][delta_dust]/332946)       #Md in solar masses
        
        if par=='Ms':
            logMsLup_det=np.log10(Lup_Tab['M*'][delta_dust])
            x=logMsLup_det
            y=np.log10(ratio)
            
        if par=='Rout':      ## Outer dust radii:
            logR_out=np.full(len(Lup_Tab),0.)
            Dp_Rout=np.full(len(Lup_Tab),0.)
            Dn_Rout=np.full(len(Lup_Tab),0.)
            i_Rad=[]
            for i in range(len(Lup_Tab)):
                if Lup_Tab['R_out'][i] != 0.:   # take the non-zero values
                    i_Rad.append(i)
            logR_out[i_Rad]=np.log10(Lup_Tab['R_out'][i_Rad])
            x=logR_out
        
        if par=='Rgas':  ## Gas radii:
            logR_gas=np.full(len(Lup_Tab),0.)
            Dp_Rgas=np.full(len(Lup_Tab),0.)
            Dn_Rgas=np.full(len(Lup_Tab),0.)
            i_Rad_gas=[]
            for i in range(len(Lup_Tab)):
                if Lup_Tab['R_gas'][i] != 0.:    # take the non-zero values
                    i_Rad_gas.append(i)
            logR_gas[i_Rad_gas]=np.log10(Lup_Tab['R_gas'][i_Rad_gas])
            x=logR_gas
    
        if par=='spidx70':    ### spectral index values from Tazzari+17:
            i_alpha70=[] 
            for i in range(len(Lup_Tab)):    # take the non-zero values
                if (Lup_Tab['alpha_J70'][i] != 0.) and (Lup_Tab['alpha_J70'][i] != -99.0):
                    i_alpha70.append(i)
            x=Lup_Tab['alpha_J70']
            err_alpha=0.1      #took 10% of uncertainties, for now, to be verified!
        if par=='spidx100':    ### spectral index values from Tazzari+17:
            i_alpha100=[] 
            for i in range(len(Lup_Tab)):    # take the non-zero values
                if (Lup_Tab['alpha_J100'][i] != 0.) and (Lup_Tab['alpha_J100'][i] != -99.0):
                    i_alpha100.append(i)
            x=Lup_Tab['alpha_J100']
            err_alpha=0.1      #took 10% of uncertainties, for now, to be verified!
        if par=='spidx160':    ### spectral index values from Tazzari+17:
            i_alpha160=[] 
            for i in range(len(Lup_Tab)):    # take the non-zero values
                if (Lup_Tab['alpha_J160'][i] != 0.) and (Lup_Tab['alpha_J160'][i] != -99.0):
                    i_alpha160.append(i)
            x=Lup_Tab['alpha_J160']
            err_alpha=0.1      #took 0.1 for uncertainties, for now, to be verified!


if survey=='ChaI':
    if n=='Mdust':
        x=ChaI_Tab['logM*'] 
        Md=ChaI_Tab['Md']
        e_Md=ChaI_Tab['e_Md']   
        y=np.log10(Md) 
        delta=ChaI_Tab['l_Md']!='<'      #observed sources for Mdisk

    if n=='Mgas':
        x=ChaI_Tab['logM*'] 
        y=Mgas_ChaI['logMgas']   
        delta=Mgas_ChaI['l_Mgas']!='<'      #observed sources for Mgas
        
    if n=='g/d':        
        delta_dust=ChaI_Tab['l_Md']!='<'     
        notdelta_dust=np.logical_not(delta_dust)
        dGas=Mgas_ChaI['l_Mgas']!='<'      #observed sources for Mgas
        delta=dGas[delta_dust]          # take only the sources detected in dust
        ratio=10**Mgas_ChaI['logMgas'][delta_dust]/(ChaI_Tab['Md'][delta_dust]/332946)    #Md in solar masses

        if par=='Ms':
            logMsCha_det=ChaI_Tab['logM*'][delta_dust]
            x=logMsCha_det
            y=np.log10(ratio)

if survey=='UppSco':
    Md=UppSco_Tab['Md_20']
    e_Md=UppSco_Tab['e_Md_20']    
    x=UppSco_Tab['logM*']
    y=np.log10(Md)  

    delta=UppSco_Tab['l_Md']!='<'      #observed sources for Mdisk

if survey=='LupChaI':
    if n=='Mgas':
        x= np.concatenate((np.array(Lup_Tab['M*']), np.array(ChaI_Tab['logM*']) )) # logMstar in solar masses
        y= np.concatenate((np.array(Mgas_Lup['logMgas']), np.array(Mgas_ChaI['logMgas']) )) # logMgas in solar masses
        dGas_ChaI=Mgas_ChaI['l_Mgas']!='<'
        dGas_Lup=Mgas_Lup['l_Mgas']!='<'
        delta=np.concatenate(( np.array(dGas_Lup), np.array(dGas_ChaI) ))
    
    if n=='g/d':
        delta_dustLup=Lup_Tab['l_Md']!='<'     
        dGasLup=Mgas_Lup['l_Mgas']!='<'      #observed sources for Mgas      
        delta_dustCha=ChaI_Tab['l_Md']!='<' 
        dGasChaI=Mgas_ChaI['l_Mgas']!='<'      #observed sources for Mgas
        ratioLup=10**Mgas_Lup['logMgas'][delta_dustLup]/(Lup_Tab['Md'][delta_dustLup]/332946)
        ratioChaI=10**Mgas_ChaI['logMgas'][delta_dustCha]/(ChaI_Tab['Md'][delta_dustCha]/332946)
        
        delta=np.concatenate((dGasLup[delta_dustLup],dGasChaI[delta_dustCha]))        
        ratio=np.concatenate((ratioLup,ratioChaI))

        if par=='Ms':
            x=np.concatenate((Column(np.log10(Lup_Tab['M*'][delta_dustLup])), 
                               Column(ChaI_Tab['logM*'][delta_dustCha]) ))    #lgMs
            y=np.log10(ratio)  #log ratio
            
            
notdelta = np.logical_not(delta)       #upperlimits


#######################################
#              Errors                #
#    Dp --> superior error bar in lg scale; Dn --> inferior error bar in log scale
if survey=='Tau': 
    if n=='Mdust':
        Dp_x=Tab_Mass['bM*_up3'][limMs]-Tab_Mass['LogM*3'][limMs]      
        Dn_x=Tab_Mass['LogM*3'][limMs]-Tab_Mass['bM*_lo3'][limMs] 
        Dp_y=Tab['Dp_20'][limMs]  
        Dn_y=Tab['Dn_20'][limMs] 
        
if survey=='Lup':
    if n=='Mdust':
        Dp_x=np.log10(Lup_Tab['M*']+Lup_Tab['err_M*'])-x  
        Dn_x=x-np.log10(Lup_Tab['M*']-Lup_Tab['err_M*'])    
        #quadratic sum of the calibration error on Md (10%)
        ecal_Md=np.sqrt((Lup_Tab['err_Md'])**2+(0.1*Lup_Tab['Md'])**2) 
        Md=Lup_Tab['Md']
        Md[notdelta]=3*Lup_Tab['err_Md'][notdelta]    # upperlimits as 3*sigma   
        Dp_y=np.log10(Md+ecal_Md)-np.log10(Md)  #superior error bar
        Dn_y=np.log10(Md)-np.log10(Md-ecal_Md)
        
    if n=='Mgas':
        Dp_x=np.log10(Lup_Tab['M*']+Lup_Tab['err_M*'])-np.log10(Lup_Tab['M*'])  
        Dn_x=np.log10(Lup_Tab['M*'])-np.log10(Lup_Tab['M*']-Lup_Tab['err_M*'])    
        Dp_y=Mgas_Lup['Dp_Mg']     
        Dn_y=Mgas_Lup['Dn_Mg']
        
    if n=='g/d':
        ecal_Md=np.sqrt((Lup_Tab['err_Md'])**2+(0.1*Lup_Tab['Md'])**2) 
        Md=Lup_Tab['Md']
        Md[notdelta_dust]=3*Lup_Tab['err_Md'][notdelta_dust]
        Dp_yMd=np.log10(Md+ecal_Md)-np.log10(Md)
        Dn_yMd=np.log10(Md)-np.log10(Md-ecal_Md)
            
        Dn_y=(Mgas_Lup['Dn_Mg'][delta_dust]/np.abs(Mgas_Lup['logMgas'][delta_dust])+
             Dn_yMd[delta_dust]/np.abs(np.log10(Md)[delta_dust]))*np.abs(np.log10(ratio))
        Dp_y=(Mgas_Lup['Dp_Mg'][delta_dust]/np.abs(Mgas_Lup['logMgas'][delta_dust])+
             Dp_yMd[delta_dust]/np.abs(np.log10(Md)[delta_dust]))*np.abs(np.log10(ratio))

        if par=='Ms':    ## errors on stellar masses
            Dp_xMs=np.log10(Lup_Tab['M*']+Lup_Tab['err_M*'])-np.log10(Lup_Tab['M*'])  
            Dn_xMs=np.log10(Lup_Tab['M*'])-np.log10(Lup_Tab['M*']-Lup_Tab['err_M*']) 
            Dp_x=Dp_xMs[delta_dust]
            Dn_x=Dn_xMs[delta_dust]
            
        if par=='Rout':    ## errors on Rout
            Dp_Rout[i_Rad]=np.log10(Lup_Tab['R_out'][i_Rad]+Lup_Tab['eR_out'][i_Rad])-logR_out[i_Rad]
            Dn_Rout[i_Rad]=logR_out[i_Rad]-np.log10(Lup_Tab['R_out'][i_Rad]-Lup_Tab['eR_out'][i_Rad])
            Dp_x=Dp_Rout
            Dn_x=Dn_Rout
        if par=='Rgas':   ## errors on Rgas.
            Dp_Rgas[i_Rad_gas]=np.log10(Lup_Tab['R_gas'][i_Rad_gas]+Lup_Tab['eR_gas'][i_Rad_gas])-logR_gas[i_Rad_gas]
            Dn_Rgas[i_Rad_gas]=logR_gas[i_Rad_gas]-np.log10(Lup_Tab['R_gas'][i_Rad_gas]-Lup_Tab['eR_gas'][i_Rad_gas])
            Dp_x=Dp_Rgas
            Dn_x=Dn_Rgas
        if par=='alpha70' or par=='alpha100' or par=='alpha160':    #errors for spidx
            Dp_x=err_alpha      
            Dn_x=err_alpha

        
if survey=='ChaI':
    if n=='Mdust':
        Dp_x=ChaI_Tab['up_logM*']-ChaI_Tab['logM*']
        Dn_x=ChaI_Tab['logM*']-ChaI_Tab['down_logM*']
        Dp_y=np.log10(Md+e_Md)-y  #superior error bar
        Dn_y=y-np.log10(Md-e_Md)   #inferior error bar
        
    if n=='Mgas':
        Dp_x=ChaI_Tab['up_logM*']-ChaI_Tab['logM*']
        Dn_x=ChaI_Tab['logM*']-ChaI_Tab['down_logM*']
        Dp_y=Mgas_ChaI['Dp_Mg']     #superior error bar in log scale
        Dn_y=Mgas_ChaI['Dn_Mg']    #inferior error bar in log scale
        
    if n=='g/d':
        Md=ChaI_Tab['Md']
        e_Md=ChaI_Tab['e_Md']   
        Dp_yMd=np.log10(Md+e_Md)-np.log10(Md)
        Dn_yMd=np.log10(Md)-np.log10(Md-e_Md)
            
        Dn_y=(Mgas_ChaI['Dn_Mg'][delta_dust]/np.abs(Mgas_ChaI['logMgas'][delta_dust])+
              Dn_yMd[delta_dust]/np.abs(np.log10(Md)[delta_dust]))*np.abs(np.log10(ratio))
        Dp_y=(Mgas_ChaI['Dp_Mg'][delta_dust]/np.abs(Mgas_ChaI['logMgas'][delta_dust])+
              Dp_yMd[delta_dust]/np.abs(np.log10(Md)[delta_dust]))*np.abs(np.log10(ratio))
        if par=='Ms':  ## errors on stellar masses
            Dp_xMs=ChaI_Tab['up_logM*']-ChaI_Tab['logM*']
            Dn_xMs=ChaI_Tab['logM*']-ChaI_Tab['down_logM*']
            Dp_x=Dp_xMs[delta_dust]
            Dn_x=Dn_xMs[delta_dust]

if survey=='UppSco':
    Dp_x=UppSco_Tab['up_logM*']
    Dn_x=UppSco_Tab['down_logM*']
    Dp_y=np.log10(Md+e_Md)-y 
    Dn_y=y-np.log10(Md-e_Md)

if survey=='LupChaI':
    if n=='Mg':
        Dp_xChaI=ChaI_Tab['up_logM*']-ChaI_Tab['logM*']
        Dn_xChaI=ChaI_Tab['logM*']-ChaI_Tab['down_logM*']
        Dp_xLup=np.log10(Lup_Tab['M*']+Lup_Tab['err_M*'])-np.log10(Lup_Tab['M*'])  
        Dn_xLup=np.log10(Lup_Tab['M*'])-np.log10(Lup_Tab['M*']-Lup_Tab['err_M*'])    
       
        Dn_x= np.concatenate(( np.array(Dn_xLup), np.array(Dn_xChaI) )) # inferior error bar on logMstar
        Dp_x= np.concatenate(( np.array(Dp_xLup), np.array(Dp_xChaI) )) # superior errorbar on logMstar 
        Dn_y= np.concatenate(( np.array(Mgas_Lup['Dn_Mg']), np.array(Mgas_ChaI['Dn_Mg']) )) 
        Dp_y= np.concatenate(( np.array(Mgas_Lup['Dp_Mg']), np.array(Mgas_ChaI['Dp_Mg']) )) 
    
    if n=='g/d':
        delta_dustLup=Lup_Tab['l_Md']!='<'     
        dGasLup=Mgas_Lup['l_Mgas']!='<'      #observed sources for Mgas      
        delta_dustCha=ChaI_Tab['l_Md']!='<' 
        dGasChaI=Mgas_ChaI['l_Mgas']!='<'      #observed sources for Mgas
        # errors for ChaI  
        Dp_yMdCha=np.log10(ChaI_Tab['Md']+ChaI_Tab['e_Md'])-np.log10(ChaI_Tab['Md'])
        Dn_yMdCha=np.log10(ChaI_Tab['Md'])-np.log10(ChaI_Tab['Md']-ChaI_Tab['e_Md'])
            
        Dny_ratioCha=(Mgas_ChaI['Dn_Mg'][delta_dustCha]/np.abs(Mgas_ChaI['logMgas'][delta_dustCha])+
              Dn_yMdCha[delta_dustCha]/np.abs(np.log10(ChaI_Tab['Md'])[delta_dustCha]))*np.abs(np.log10(ratioChaI))
        Dpy_ratioCha=(Mgas_ChaI['Dp_Mg'][delta_dustCha]/np.abs(Mgas_ChaI['logMgas'][delta_dustCha])+
              Dp_yMdCha[delta_dustCha]/np.abs(np.log10(ChaI_Tab['Md'][delta_dustCha])))*np.abs(np.log10(ratioChaI))
        # errors for Lup
        ecal_MdLup=np.sqrt((Lup_Tab['err_Md'])**2+(0.1*Lup_Tab['Md'])**2) 
        MdLup=Lup_Tab['Md']
        MdLup[np.logical_not(delta_dustLup)]=3*Lup_Tab['err_Md'][np.logical_not(delta_dustLup)]
        Dp_yMdLup=np.log10(MdLup+ecal_MdLup)-np.log10(MdLup)
        Dn_yMdLup=np.log10(MdLup)-np.log10(MdLup-ecal_MdLup)
            
        Dny_ratioLup=(Mgas_Lup['Dn_Mg'][delta_dustLup]/np.abs(Mgas_Lup['logMgas'][delta_dustLup])+
             Dn_yMdLup[delta_dustLup]/np.abs(np.log10(MdLup)[delta_dustLup]))*np.abs(np.log10(ratioLup))
        Dpy_ratioLup=(Mgas_Lup['Dp_Mg'][delta_dustLup]/np.abs(Mgas_Lup['logMgas'][delta_dustLup])+
             Dp_yMdLup[delta_dustLup]/np.abs(np.log10(MdLup)[delta_dustLup]))*np.abs(np.log10(ratioLup))
        
        if par=='Ms':
            Dp_xMsCha=ChaI_Tab['up_logM*']-ChaI_Tab['logM*']
            Dn_xMsCha=ChaI_Tab['logM*']-ChaI_Tab['down_logM*']
            Dp_xMsLup=np.log10(Lup_Tab['M*']+Lup_Tab['err_M*'])-np.log10(Lup_Tab['M*'])  
            Dn_xMsLup=np.log10(Lup_Tab['M*'])-np.log10(Lup_Tab['M*']-Lup_Tab['err_M*']) 
            
            #combine Lup and Cha data
            Dn_y=np.concatenate((Column(Dny_ratioLup),Column(Dny_ratioCha)))
            Dp_y=np.concatenate((Column(Dpy_ratioLup),Column(Dpy_ratioCha)))
            Dn_x=np.concatenate((Column(Dn_xMsLup[delta_dustLup]), Column(Dn_xMsCha[delta_dustCha])))
            Dp_x=np.concatenate((Column(Dp_xMsLup[delta_dustLup]), Column(Dp_xMsCha[delta_dustCha])))
        


#### Run the Linmix code for the Bayesian linear regression analysis #

In [28]:
# Define the errors on the variables (The linMix code assumes Gaussian measurement errors)
xsig=(Dp_x+Dn_x)/2.
ysig=(Dp_y+Dn_y)/2.

In [13]:
####run the linmix on the *DETECTED* object in order to derive the dispersion###

lm_upp_det = linmix.LinMix(x[delta], y[delta], xsig=xsig[delta], ysig=ysig[delta], 
                           delta=delta[delta], K=2, nchains=20)  
lm_upp_det.run_mcmc(miniter=5000, maxiter=100000, silent=True)    	#run the Markov Chain Monte Carlo  

In [14]:
#### set the upper limit uncertanties equal to the dispersion of detections #####
ysig[notdelta]=np.sqrt(lm_upp_det.chain['sigsqr'].mean())

In [15]:
#### run the linmix on the full sample consider the upper limits with 100 MC runs #####
lm_upp = linmix.LinMix(x, y, xsig=xsig, ysig=ysig, delta=delta, K=2, nchains=100)  
lm_upp.run_mcmc(miniter=5000, maxiter=100000, silent=True)    #run the Markov Chain Monte Carlo  

In [16]:
#    writing output of regression analysis in an ascii file    #

if survey=='Tau':
    if n=='Mdust':
        if T=='var':
            ascii.write(lm_upp.chain[['alpha', 'beta', 'sigsqr','mu0', 'usqr', 'wsqr','ximean', 'xisig', 'corr']],
                        'Md_vs_Ms.pyout', overwrite=True)
        if T=='20':
            ascii.write(lm_upp.chain[['alpha', 'beta', 'sigsqr','mu0', 'usqr', 'wsqr','ximean', 'xisig', 'corr']],
                        'Md_20_vs_Ms.pyout', overwrite=True)
            
if survey=='Lup':
    if n=='Mdust':
        ascii.write(lm_upp.chain[['alpha', 'beta', 'sigsqr','mu0', 'usqr', 'wsqr','ximean', 'xisig', 'corr']],
                        'Lup_Md_vs_Ms.pyout', overwrite=True)
    if n=='Mgas':
        ascii.write(lm_upp.chain[['alpha', 'beta', 'sigsqr','mu0', 'usqr', 'wsqr','ximean', 'xisig', 'corr']],
                    'Lup_Mg_vs_Ms.pyout', overwrite=True)
    if n=='g/d':
        if par=='Ms':
            ascii.write(lm_upp.chain[['alpha', 'beta', 'sigsqr','mu0', 'usqr', 'wsqr','ximean', 'xisig',
                                               'corr']], 'ratioLup_vs_Ms.pyout', overwrite=True)

        
if survey=='ChaI':
    if n=='Mdust':
        ascii.write(lm_upp.chain[['alpha', 'beta', 'sigsqr','mu0', 'usqr', 'wsqr','ximean', 'xisig', 'corr']],
                    'ChaI_Md_vs_Ms.pyout', overwrite=True)
    if n=='Mgas':
        ascii.write(lm_upp.chain[['alpha', 'beta', 'sigsqr','mu0', 'usqr', 'wsqr','ximean', 'xisig', 'corr']],
                    'ChaI_Mg_vs_Ms.pyout', overwrite=True)
    if n=='g/d':
        ascii.write(lm_upp.chain[['alpha', 'beta', 'sigsqr','mu0', 'usqr', 'wsqr','ximean', 'xisig', 'corr']],
                    'ratioCha_vs_Ms.pyout', overwrite=True)
        
if survey=='UppSco':
    ascii.write(lm_upp.chain[['alpha', 'beta', 'sigsqr','mu0', 'usqr', 'wsqr','ximean', 'xisig', 'corr']],
                    'UppSco_Md_vs_Ms.pyout', overwrite=True)
    
if survey=='LupChaI':
    if n=='Mgas':
        ascii.write(lm_upp.chain[['alpha', 'beta', 'sigsqr','mu0', 'usqr', 'wsqr','ximean', 'xisig', 'corr']],
                    'LupCha_Mg_vs_Ms.pyout', overwrite=True)
    if n=='g/d':
        if par=='Ms':
            ascii.write(lm_upp.chain[['alpha', 'beta', 'sigsqr','mu0', 'usqr', 'wsqr','ximean', 'xisig', 'corr']],
                    'ratioLupCha_vs_Ms.pyout', overwrite=True)


In [31]:
#reading output of regression analysis
if survey=='Tau':
    pyout = ascii.read('Md_20_vs_Ms.pyout')
    
if survey=='Lup':
    if n=='Mdust':
        pyout = ascii.read('Lup_Md_vs_Ms.pyout')
    if n=='Mgas':
        pyout=ascii.read('Lup_Mg_vs_Ms.pyout')
    if n=='g/d':
        if par=='Ms':
            pyout=ascii.read('ratioLup_vs_Ms.pyout')

if survey=='ChaI':
    if n=='Mdust':
        pyout = ascii.read('ChaI_Md_vs_Ms.pyout')
    if n=='Mgas':
        pyout=ascii.read('ChaI_Mg_vs_Ms.pyout')
    if n=='g/d':
        if par=='Ms':
            pyout=ascii.read('ratioCha_vs_Ms.pyout')
        
if survey=='UppSco':
    pyout = ascii.read('UppSco_Md_vs_Ms.pyout')
    
if survey=='LupChaI':
    if n=='Mgas':
        pyout=ascii.read('LupCha_Mg_vs_Ms.pyout')
    if n=='g/d':
        if par=='Ms':
            pyout=ascii.read('ratioLupCha_vs_Ms.pyout')

a=pyout['alpha']
b=pyout['beta']
sig=np.sqrt((pyout['sigsqr']))
corr=pyout['corr']

print a.mean(), a.std()
print b.mean(), b.std()
print sig.mean(), sig.std()
print corr.mean(), corr.std()

0.136478806849 0.114518649993
-0.119845124089 0.21701365807
0.609495735472 0.0697376760125
-0.0707016801614 0.12486758539
